In [1]:
## This file implements neural networks and random forest on p11presabsSTCC_qual.
## Due to the imbalanced data, we implement both over-sampling method and the combination of under- and over-sampling method.
## For fully-connected neural networks, the accuracy is 90.24% for over-sampling data, 91.46% after improvement.
## And it is 95.70% for combination data and 94.62% with dropout and regularization.
## For random forest, the accuracy is 90.85% for over-sampling data and 92.47% for combination data.
## For random forest with cross-validation, the mean accuracy is 85.92% for over-sampling, and 95.78% for combination data.

In [1]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p11presabsSTCC_qual.csv')
df.shape

(255, 976)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      2
1      1
2      2
3      2
4      2
5      2
6      2
7      2
8      2
9      2
10     1
11     2
12     2
13     1
14     2
15     2
16     0
17     0
18     2
19     0
20     0
21     0
22     1
23     2
24     2
25     0
26     2
27     0
28     2
29     2
      ..
225    2
226    1
227    2
228    2
229    2
230    1
231    2
232    2
233    2
234    2
235    2
236    2
237    2
238    2
239    2
240    2
241    2
242    0
243    2
244    2
245    2
246    2
247    2
248    2
249    2
250    2
251    1
252    2
253    2
254    2
Name: pheno, Length: 255, dtype: int64

In [5]:
df['pheno'].value_counts()

2    182
1     48
0     25
Name: pheno, dtype: int64

In [6]:
df.head()

,id,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,...,group_1300,group_1972,group_2563,group_2881,group_7076,group_8027,group_892,ST,CC,pheno
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,8,8,1
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
3,120335,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
4,120337,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2


In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(255, 975)

In [9]:
df_clean.head()

,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,TTTTTAGGTAAGG,...,group_1300,group_1972,group_2563,group_2881,group_7076,group_8027,group_892,ST,CC,pheno
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,8,8,1
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2


In [10]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 974) (255,)


In [11]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 182), (1, 182), (2, 182)]


Using TensorFlow backend.


In [12]:
# combination of under- and over- sampling
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 131), (1, 111), (2, 67)]


In [13]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [14]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [15]:
############# Fully-Connected Neural Network ################

In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [17]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [18]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 382 samples, validate on 164 samples
Epoch 1/100
382/382 [==============================] - 0s 326us/step - loss: 3.5645 - accuracy: 0.3639 - val_loss: 1.4567 - val_accuracy: 0.4939
Epoch 2/100
382/382 [==============================] - 0s 95us/step - loss: 1.5241 - accuracy: 0.5157 - val_loss: 0.7876 - val_accuracy: 0.6707
Epoch 3/100
382/382 [==============================] - 0s 97us/step - loss: 1.2026 - accuracy: 0.6099 - val_loss: 0.8726 - val_accuracy: 0.6951
Epoch 4/100
382/382 [==============================] - 0s 93us/step - loss: 1.0780 - accuracy: 0.6387 - val_loss: 0.7274 - val_accuracy: 0.6890
Epoch 5/100
382/382 [==============================] - 0s 99us/step - loss: 0.7765 - accuracy: 0.6780 - val_loss: 0.6786 - val_accuracy: 0.6768
Epoch 6/100
382/382 [==============================] - 0s 96us/step - loss: 0.6743 - accuracy: 0.7042 - val_loss: 0.6151 - val_accuracy: 0.7134
Epoch 7/100
382/382 [==============================] - 0s 97us/step - loss: 0.6335 - accu

382/382 [==============================] - 0s 107us/step - loss: 0.1768 - accuracy: 0.9450 - val_loss: 0.3264 - val_accuracy: 0.8902
Epoch 58/100
382/382 [==============================] - 0s 98us/step - loss: 0.1844 - accuracy: 0.9372 - val_loss: 0.3455 - val_accuracy: 0.8659
Epoch 59/100
382/382 [==============================] - 0s 83us/step - loss: 0.2052 - accuracy: 0.9136 - val_loss: 0.3679 - val_accuracy: 0.8841
Epoch 60/100
382/382 [==============================] - 0s 81us/step - loss: 0.2734 - accuracy: 0.9058 - val_loss: 0.3381 - val_accuracy: 0.8659
Epoch 61/100
382/382 [==============================] - 0s 77us/step - loss: 0.1906 - accuracy: 0.9241 - val_loss: 0.3227 - val_accuracy: 0.9085
Epoch 62/100
382/382 [==============================] - 0s 76us/step - loss: 0.2423 - accuracy: 0.9110 - val_loss: 0.3278 - val_accuracy: 0.9085
Epoch 63/100
382/382 [==============================] - 0s 79us/step - loss: 0.2391 - accuracy: 0.9267 - val_loss: 0.3446 - val_accuracy: 0.88

In [20]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

164/164 [==============================] - 0s 81us/step
over-sampling test accuracy: 90.24%


In [25]:
#### improve neural network on over-sampling data with dropout
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [26]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [27]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 382 samples, validate on 164 samples
Epoch 1/1000
382/382 [==============================] - 0s 507us/step - loss: 4.4824 - accuracy: 0.3796 - val_loss: 2.0346 - val_accuracy: 0.4390
Epoch 2/1000
382/382 [==============================] - 0s 99us/step - loss: 4.2909 - accuracy: 0.4398 - val_loss: 1.1503 - val_accuracy: 0.5427
Epoch 3/1000
382/382 [==============================] - 0s 106us/step - loss: 3.1706 - accuracy: 0.5576 - val_loss: 1.1159 - val_accuracy: 0.6037
Epoch 4/1000
382/382 [==============================] - 0s 101us/step - loss: 3.3096 - accuracy: 0.5209 - val_loss: 0.9464 - val_accuracy: 0.6159
Epoch 5/1000
382/382 [==============================] - 0s 115us/step - loss: 3.3850 - accuracy: 0.5838 - val_loss: 0.7825 - val_accuracy: 0.6220
Epoch 6/1000
382/382 [==============================] - 0s 103us/step - loss: 3.8014 - accuracy: 0.5471 - val_loss: 0.8384 - val_accuracy: 0.6707
Epoch 7/1000
382/382 [==============================] - 0s 104us/step - loss: 3

382/382 [==============================] - 0s 108us/step - loss: 1.7961 - accuracy: 0.7042 - val_loss: 1.0192 - val_accuracy: 0.7561
Epoch 57/1000
382/382 [==============================] - 0s 168us/step - loss: 1.9369 - accuracy: 0.7147 - val_loss: 1.1879 - val_accuracy: 0.7683
Epoch 58/1000
382/382 [==============================] - 0s 150us/step - loss: 1.8608 - accuracy: 0.7042 - val_loss: 0.9895 - val_accuracy: 0.8476
Epoch 59/1000
382/382 [==============================] - 0s 101us/step - loss: 2.0589 - accuracy: 0.6990 - val_loss: 1.1649 - val_accuracy: 0.8354
Epoch 60/1000
382/382 [==============================] - 0s 101us/step - loss: 1.9553 - accuracy: 0.7225 - val_loss: 1.2056 - val_accuracy: 0.7805
Epoch 61/1000
382/382 [==============================] - 0s 92us/step - loss: 1.9897 - accuracy: 0.6885 - val_loss: 1.3150 - val_accuracy: 0.7988
Epoch 62/1000
382/382 [==============================] - 0s 96us/step - loss: 1.6343 - accuracy: 0.7304 - val_loss: 1.1527 - val_accu

382/382 [==============================] - 0s 89us/step - loss: 1.6184 - accuracy: 0.7304 - val_loss: 1.1991 - val_accuracy: 0.8232
Epoch 113/1000
382/382 [==============================] - 0s 92us/step - loss: 1.2892 - accuracy: 0.7565 - val_loss: 1.1935 - val_accuracy: 0.8476
Epoch 114/1000
382/382 [==============================] - 0s 85us/step - loss: 1.6239 - accuracy: 0.7277 - val_loss: 1.2479 - val_accuracy: 0.8415
Epoch 115/1000
382/382 [==============================] - 0s 83us/step - loss: 1.1829 - accuracy: 0.7592 - val_loss: 1.0567 - val_accuracy: 0.8780
Epoch 116/1000
382/382 [==============================] - 0s 82us/step - loss: 1.1563 - accuracy: 0.7565 - val_loss: 0.8941 - val_accuracy: 0.8720
Epoch 117/1000
382/382 [==============================] - 0s 82us/step - loss: 1.2725 - accuracy: 0.7356 - val_loss: 1.0945 - val_accuracy: 0.8415
Epoch 118/1000
382/382 [==============================] - 0s 93us/step - loss: 1.0628 - accuracy: 0.7801 - val_loss: 0.9014 - val_acc

Epoch 168/1000
382/382 [==============================] - 0s 84us/step - loss: 1.6815 - accuracy: 0.6728 - val_loss: 1.1311 - val_accuracy: 0.8537
Epoch 169/1000
382/382 [==============================] - 0s 91us/step - loss: 1.4995 - accuracy: 0.7199 - val_loss: 1.3494 - val_accuracy: 0.8476
Epoch 170/1000
382/382 [==============================] - 0s 80us/step - loss: 1.4963 - accuracy: 0.7304 - val_loss: 1.0651 - val_accuracy: 0.8720
Epoch 171/1000
382/382 [==============================] - 0s 81us/step - loss: 0.9930 - accuracy: 0.7277 - val_loss: 0.9453 - val_accuracy: 0.8537
Epoch 172/1000
382/382 [==============================] - 0s 82us/step - loss: 1.2199 - accuracy: 0.7173 - val_loss: 1.4229 - val_accuracy: 0.8415
Epoch 173/1000
382/382 [==============================] - 0s 86us/step - loss: 1.1537 - accuracy: 0.7670 - val_loss: 0.9957 - val_accuracy: 0.8659
Epoch 174/1000
382/382 [==============================] - 0s 80us/step - loss: 1.2621 - accuracy: 0.7382 - val_loss: 1

382/382 [==============================] - 0s 83us/step - loss: 0.8770 - accuracy: 0.7853 - val_loss: 0.8889 - val_accuracy: 0.8963
Epoch 224/1000
382/382 [==============================] - 0s 99us/step - loss: 0.9663 - accuracy: 0.7958 - val_loss: 0.8019 - val_accuracy: 0.8963
Epoch 225/1000
382/382 [==============================] - 0s 82us/step - loss: 0.6426 - accuracy: 0.7801 - val_loss: 0.8542 - val_accuracy: 0.8720
Epoch 226/1000
382/382 [==============================] - 0s 82us/step - loss: 1.0061 - accuracy: 0.7565 - val_loss: 0.8522 - val_accuracy: 0.8780
Epoch 227/1000
382/382 [==============================] - 0s 79us/step - loss: 0.7855 - accuracy: 0.7723 - val_loss: 0.8137 - val_accuracy: 0.9024
Epoch 228/1000
382/382 [==============================] - 0s 82us/step - loss: 0.8913 - accuracy: 0.7592 - val_loss: 0.7598 - val_accuracy: 0.8841
Epoch 229/1000
382/382 [==============================] - 0s 85us/step - loss: 0.9406 - accuracy: 0.7775 - val_loss: 1.0064 - val_acc

...

382/382 [==============================] - 0s 85us/step - loss: 0.7258 - accuracy: 0.7880 - val_loss: 0.9760 - val_accuracy: 0.9207
Epoch 724/1000
382/382 [==============================] - 0s 87us/step - loss: 0.6288 - accuracy: 0.8010 - val_loss: 1.0375 - val_accuracy: 0.9146
Epoch 725/1000
382/382 [==============================] - 0s 82us/step - loss: 1.3344 - accuracy: 0.7408 - val_loss: 1.8659 - val_accuracy: 0.8720
Epoch 726/1000
382/382 [==============================] - 0s 83us/step - loss: 1.8576 - accuracy: 0.6780 - val_loss: 1.8632 - val_accuracy: 0.8780
Epoch 727/1000
382/382 [==============================] - 0s 94us/step - loss: 1.7638 - accuracy: 0.7487 - val_loss: 1.8123 - val_accuracy: 0.8780
Epoch 728/1000
382/382 [==============================] - 0s 86us/step - loss: 1.5806 - accuracy: 0.7513 - val_loss: 2.2537 - val_accuracy: 0.8354
Epoch 729/1000
382/382 [==============================] - 0s 85us/step - loss: 1.7344 - accuracy: 0.7251 - val_loss: 1.9364 - val_acc

Epoch 779/1000
382/382 [==============================] - 0s 88us/step - loss: 1.2518 - accuracy: 0.7618 - val_loss: 1.5764 - val_accuracy: 0.9024
Epoch 780/1000
382/382 [==============================] - 0s 85us/step - loss: 1.2389 - accuracy: 0.7487 - val_loss: 1.6810 - val_accuracy: 0.8720
Epoch 781/1000
382/382 [==============================] - 0s 82us/step - loss: 1.2516 - accuracy: 0.7513 - val_loss: 1.5466 - val_accuracy: 0.8963
Epoch 782/1000
382/382 [==============================] - 0s 82us/step - loss: 1.2313 - accuracy: 0.7801 - val_loss: 2.0841 - val_accuracy: 0.8354
Epoch 783/1000
382/382 [==============================] - 0s 81us/step - loss: 1.5333 - accuracy: 0.7016 - val_loss: 1.5409 - val_accuracy: 0.8963
Epoch 784/1000
382/382 [==============================] - 0s 81us/step - loss: 1.4570 - accuracy: 0.7173 - val_loss: 1.5971 - val_accuracy: 0.8780
Epoch 785/1000
382/382 [==============================] - 0s 84us/step - loss: 1.3020 - accuracy: 0.7251 - val_loss: 1

382/382 [==============================] - 0s 85us/step - loss: 1.5768 - accuracy: 0.7435 - val_loss: 1.5814 - val_accuracy: 0.8963
Epoch 835/1000
382/382 [==============================] - 0s 87us/step - loss: 1.5083 - accuracy: 0.7592 - val_loss: 1.3985 - val_accuracy: 0.8963
Epoch 836/1000
382/382 [==============================] - 0s 85us/step - loss: 1.4309 - accuracy: 0.7016 - val_loss: 1.5357 - val_accuracy: 0.8963
Epoch 837/1000
382/382 [==============================] - 0s 83us/step - loss: 1.4866 - accuracy: 0.7277 - val_loss: 1.5695 - val_accuracy: 0.8841
Epoch 838/1000
382/382 [==============================] - 0s 84us/step - loss: 1.2906 - accuracy: 0.7618 - val_loss: 1.6106 - val_accuracy: 0.8902
Epoch 839/1000
382/382 [==============================] - 0s 84us/step - loss: 1.3118 - accuracy: 0.7435 - val_loss: 1.6550 - val_accuracy: 0.8780
Epoch 840/1000
382/382 [==============================] - 0s 82us/step - loss: 1.5343 - accuracy: 0.7592 - val_loss: 1.7468 - val_acc

Epoch 890/1000
382/382 [==============================] - 0s 87us/step - loss: 1.2412 - accuracy: 0.7749 - val_loss: 1.6255 - val_accuracy: 0.8780
Epoch 891/1000
382/382 [==============================] - 0s 87us/step - loss: 1.1654 - accuracy: 0.7251 - val_loss: 1.5950 - val_accuracy: 0.8963
Epoch 892/1000
382/382 [==============================] - 0s 84us/step - loss: 1.3249 - accuracy: 0.7356 - val_loss: 1.5777 - val_accuracy: 0.9024
Epoch 893/1000
382/382 [==============================] - 0s 83us/step - loss: 1.3089 - accuracy: 0.7592 - val_loss: 1.6077 - val_accuracy: 0.8963
Epoch 894/1000
382/382 [==============================] - 0s 84us/step - loss: 1.2915 - accuracy: 0.7225 - val_loss: 1.5662 - val_accuracy: 0.8963
Epoch 895/1000
382/382 [==============================] - 0s 84us/step - loss: 1.3046 - accuracy: 0.7173 - val_loss: 1.6071 - val_accuracy: 0.8963
Epoch 896/1000
382/382 [==============================] - 0s 82us/step - loss: 1.2937 - accuracy: 0.7487 - val_loss: 1

Epoch 946/1000
382/382 [==============================] - 0s 85us/step - loss: 1.0496 - accuracy: 0.7539 - val_loss: 1.6737 - val_accuracy: 0.8841
Epoch 947/1000
382/382 [==============================] - 0s 86us/step - loss: 1.3275 - accuracy: 0.7251 - val_loss: 1.4989 - val_accuracy: 0.9024
Epoch 948/1000
382/382 [==============================] - 0s 83us/step - loss: 1.3202 - accuracy: 0.7251 - val_loss: 1.4977 - val_accuracy: 0.9024
Epoch 949/1000
382/382 [==============================] - 0s 84us/step - loss: 1.5613 - accuracy: 0.7225 - val_loss: 1.7845 - val_accuracy: 0.8537
Epoch 950/1000
382/382 [==============================] - 0s 83us/step - loss: 1.4084 - accuracy: 0.7356 - val_loss: 1.3867 - val_accuracy: 0.8963
Epoch 951/1000
382/382 [==============================] - 0s 84us/step - loss: 1.6659 - accuracy: 0.6990 - val_loss: 1.6914 - val_accuracy: 0.8659
Epoch 952/1000
382/382 [==============================] - 0s 83us/step - loss: 1.3320 - accuracy: 0.7408 - val_loss: 1

In [28]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

164/164 [==============================] - 0s 100us/step
over-sampling test accuracy: 91.46%


In [29]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(3, activation='softmax'),
])

In [30]:
model1_comb.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 216 samples, validate on 93 samples
Epoch 1/100
216/216 [==============================] - 0s 487us/step - loss: 1.1095 - accuracy: 0.6389 - val_loss: 0.5169 - val_accuracy: 0.8817
Epoch 2/100
216/216 [==============================] - 0s 123us/step - loss: 0.5642 - accuracy: 0.8287 - val_loss: 0.4032 - val_accuracy: 0.8817
Epoch 3/100
216/216 [==============================] - 0s 138us/step - loss: 0.4504 - accuracy: 0.8565 - val_loss: 0.3595 - val_accuracy: 0.9032
Epoch 4/100
216/216 [==============================] - 0s 137us/step - loss: 0.3905 - accuracy: 0.8704 - val_loss: 0.3258 - val_accuracy: 0.9140
Epoch 5/100
216/216 [==============================] - 0s 125us/step - loss: 0.3342 - accuracy: 0.9120 - val_loss: 0.2962 - val_accuracy: 0.9032
Epoch 6/100
216/216 [==============================] - 0s 130us/step - loss: 0.2770 - accuracy: 0.9306 - val_loss: 0.2670 - val_accuracy: 0.9140
Epoch 7/100
216/216 [==============================] - 0s 124us/step - loss: 0.2460 -

Epoch 57/100
216/216 [==============================] - 0s 133us/step - loss: 0.0491 - accuracy: 0.9769 - val_loss: 0.1307 - val_accuracy: 0.9570
Epoch 58/100
216/216 [==============================] - 0s 129us/step - loss: 0.0479 - accuracy: 0.9907 - val_loss: 0.1266 - val_accuracy: 0.9570
Epoch 59/100
216/216 [==============================] - 0s 104us/step - loss: 0.0473 - accuracy: 0.9907 - val_loss: 0.1218 - val_accuracy: 0.9570
Epoch 60/100
216/216 [==============================] - 0s 104us/step - loss: 0.0454 - accuracy: 0.9907 - val_loss: 0.1354 - val_accuracy: 0.9570
Epoch 61/100
216/216 [==============================] - 0s 104us/step - loss: 0.0447 - accuracy: 0.9907 - val_loss: 0.1227 - val_accuracy: 0.9570
Epoch 62/100
216/216 [==============================] - 0s 100us/step - loss: 0.0438 - accuracy: 0.9907 - val_loss: 0.1241 - val_accuracy: 0.9570
Epoch 63/100
216/216 [==============================] - 0s 111us/step - loss: 0.0428 - accuracy: 0.9907 - val_loss: 0.1239 -

In [32]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

93/93 [==============================] - 0s 125us/step
combination test accuracy: 95.70%


In [37]:
#### add dropout and regularization
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [38]:
model1_comb2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [39]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 216 samples, validate on 93 samples
Epoch 1/100
216/216 [==============================] - 0s 829us/step - loss: 5.9042 - accuracy: 0.4167 - val_loss: 2.0250 - val_accuracy: 0.5484
Epoch 2/100
216/216 [==============================] - 0s 117us/step - loss: 4.5419 - accuracy: 0.5324 - val_loss: 1.3719 - val_accuracy: 0.8280
Epoch 3/100
216/216 [==============================] - 0s 126us/step - loss: 4.1702 - accuracy: 0.6898 - val_loss: 1.1807 - val_accuracy: 0.8172
Epoch 4/100
216/216 [==============================] - 0s 137us/step - loss: 4.2690 - accuracy: 0.6574 - val_loss: 1.0627 - val_accuracy: 0.8387
Epoch 5/100
216/216 [==============================] - 0s 135us/step - loss: 3.3531 - accuracy: 0.6806 - val_loss: 1.0260 - val_accuracy: 0.8495
Epoch 6/100
216/216 [==============================] - 0s 191us/step - loss: 3.6435 - accuracy: 0.7083 - val_loss: 1.0100 - val_accuracy: 0.8495
Epoch 7/100
216/216 [==============================] - 0s 126us/step - loss: 3.7846 -

Epoch 57/100
216/216 [==============================] - 0s 149us/step - loss: 1.9764 - accuracy: 0.7639 - val_loss: 1.0976 - val_accuracy: 0.9355
Epoch 58/100
216/216 [==============================] - 0s 114us/step - loss: 2.1643 - accuracy: 0.7639 - val_loss: 1.0633 - val_accuracy: 0.9355
Epoch 59/100
216/216 [==============================] - 0s 114us/step - loss: 2.1900 - accuracy: 0.7454 - val_loss: 1.0172 - val_accuracy: 0.9355
Epoch 60/100
216/216 [==============================] - 0s 106us/step - loss: 2.0983 - accuracy: 0.7778 - val_loss: 1.0325 - val_accuracy: 0.9247
Epoch 61/100
216/216 [==============================] - 0s 108us/step - loss: 1.7762 - accuracy: 0.8102 - val_loss: 1.0832 - val_accuracy: 0.9355
Epoch 62/100
216/216 [==============================] - 0s 103us/step - loss: 1.8419 - accuracy: 0.7778 - val_loss: 1.0364 - val_accuracy: 0.9355
Epoch 63/100
216/216 [==============================] - 0s 101us/step - loss: 1.8813 - accuracy: 0.7824 - val_loss: 1.0257 -

In [40]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

93/93 [==============================] - 0s 92us/step
combination test accuracy: 94.62%


In [41]:
############## Random Forest ##############

In [42]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [43]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 90.85%


In [44]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [45]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 92.47%


In [46]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.76923077 0.84615385 0.93421053 0.81333333 0.93333333]
0.8592523616734142


In [47]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rfcv_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[0.97777778 0.97727273 0.97674419 0.92857143 0.92857143]
0.9577875096479748
